## Introdução ao Aprendizado de Máquina
Hoje vamos aprender alguns conceitos básicos sobre aprendizado de máquina como pré-processamento e validação de experimentos.

## Abrindo e Pré- Processando o Data-set

## Escolhendo O classificador

## Analisando os Resultados